In [60]:
#market = Ticker(['fb', 'aapl', 'amzn', 'nflx'], asynchronous=True)

In [61]:
#import pandas_datareader as pdr
#nasdaqSymbols=pdr.get_nasdaq_symbols()['NASDAQ Symbol'].to_numpy().tolist()

In [1]:
from yahooquery import Ticker
import pandas as pd
import bs4 as bs
import pickle
import requests

def save_russell1000_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/Russell_1000_Index')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[1].text.rstrip()
        tickers.append(ticker)
        
    with open("russell1000tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

market = Ticker(save_russell1000_tickers(), asynchronous=True)

In [104]:
#balanceSheets.shape #this call doesn't return all russell 1000 tickers, b/c of missing fields
#Ticker('MET').balance_sheet() #missing CurrentDeferredLiabilities OtherNonCurrentLiabilities hence the prev statement fails

(1290, 140)

In [12]:
#balanceSheets.loc['BIO']
#balanceSheets.loc['MET']
#incomeStatements.shape
#valuationMeasures = valuationMeasures[valuationMeasures['MarketCap']>100000000000]
valuationMeasures.__dict__


{'_is_copy': <weakref at 0x12950b410; dead>,
 '_data': BlockManager
 Items: Index(['MarketCap'], dtype='object')
 Axis 1: Index(['ABT', 'ABBV', 'MMM', 'ACN', 'ADBE', 'AMD', 'GOOGL', 'GOOG', 'AMZN',
        'AXP', 'AMT', 'AMGN', 'ANTM', 'AAPL', 'AMAT', 'T', 'TEAM', 'BAC', 'BLK',
        'BA', 'BKNG', 'BMY', 'AVGO', 'CAT', 'CHTR', 'CVX', 'CSCO', 'C', 'KO',
        'CMCSA', 'COST', 'CVS', 'DHR', 'DE', 'DIS', 'EL', 'XOM', 'GE', 'GS',
        'HON', 'HD', 'INTC', 'IBM', 'INTU', 'JNJ', 'JPM', 'LLY', 'LOW', 'MA',
        'MCD', 'MDT', 'MRK', 'FB', 'MSFT', 'MS', 'NFLX', 'NVDA', 'ORCL', 'NEE',
        'PYPL', 'NKE', 'PEP', 'PFE', 'PM', 'PG', 'PLD', 'QCOM', 'SPGI', 'CRM',
        'SCHW', 'RTX', 'NOW', 'SQ', 'SBUX', 'TGT', 'TXN', 'TSLA', 'TMO', 'TMUS',
        'UNP', 'UPS', 'UNH', 'VZ', 'WMT', 'WFC', 'V', 'ZTS'],
       dtype='object', name='symbol')
 FloatBlock: slice(0, 1, 1), 1 x 87, dtype: float64,
 '_item_cache': {},
 '_attrs': {}}

In [70]:
#valuationMeasures=market.valuation_measures[['asOfDate','MarketCap','periodType']]
#valuationMeasures=valuationMeasures[valuationMeasures["periodType"]=="TTM"]
#valuationMeasures=valuationMeasures.groupby(['symbol'])['MarketCap'].max().to_frame() #this is problematic as it is not necessarily the latest

In [2]:
valuationMeasures = pd.DataFrame.from_dict(market.summary_detail).loc[['marketCap']].transpose()
valuationMeasures.index.name = 'symbol'
valuationMeasures.columns=['MarketCap']
#valuationMeasures.MarketCap = pd.to_numeric(valuationMeasures.MarketCap, errors='coerce')
#100000mm market cap limit
#valuationMeasures = valuationMeasures[valuationMeasures['MarketCap']>100000000000]

In [ ]:
# join marketcap to reduce rows
filteredBalancedSheets = pd.merge(valuationMeasures, market.balance_sheet(), on=['symbol']).fillna(0)
filteredIncomeStatements = pd.merge(valuationMeasures, market.income_statement(), on=['symbol']).fillna(0)

In [61]:
# roll up balanced sheets
balanceSheets=filteredBalancedSheets[filteredBalancedSheets['periodType']=="12M"].sort_values('asOfDate').groupby('symbol').tail(1)
balanceSheets=balanceSheets[[
    'asOfDate',
    'periodType',
    'InvestedCapital',
    'CashAndCashEquivalents',
    'CurrentDeferredLiabilities',
    'LongTermDebtAndCapitalLeaseObligation',
    'NonCurrentDeferredLiabilities',
    'OtherNonCurrentLiabilities',
    'CapitalStock',
    'CommonStock'
]]
balanceSheets.shape

In [ ]:
# roll up income statements
incomeStatements=filteredIncomeStatements[filteredIncomeStatements['periodType']=="12M"].sort_values('asOfDate').groupby('symbol').tail(1)
incomeStatements=incomeStatements[[
    'asOfDate',
    'periodType',
    'EBIT'
]]
incomeStatements.shape

In [115]:
mergedBsIs = pd.merge(balanceSheets, incomeStatements, on=['asOfDate','symbol','periodType'])

In [63]:
#mergedBsIsSum = pd.merge(mergedBsIs, valuationMeasures, on=['symbol']).fillna(0)

In [64]:
#may not be nec
#mergedBsIsSum = mergedBsIsSum.groupby(['symbol'])['asOfDate',
                                  'InvestedCapital',
                                  'CashAndCashEquivalents',
                                  'CurrentDeferredLiabilities',
                                  'LongTermDebtAndCapitalLeaseObligation',
                                  'NonCurrentDeferredLiabilities',
                                  'OtherNonCurrentLiabilities',
                                  'CapitalStock',
                                  'CommonStock',
                                  'EBIT',
                                  'MarketCap'].max()

/Users/vincentchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [65]:
roic = mergedBsIsSum.EBIT / mergedBsIsSum.InvestedCapital

In [66]:
#mergedBsIsSum.EBIT

In [67]:
#roic

In [68]:
cash = mergedBsIsSum.CashAndCashEquivalents

In [69]:
#cash

In [70]:
debt = mergedBsIsSum.CurrentDeferredLiabilities+mergedBsIsSum.LongTermDebtAndCapitalLeaseObligation+mergedBsIsSum.NonCurrentDeferredLiabilities+mergedBsIsSum.OtherNonCurrentLiabilities

In [71]:
#debt

In [72]:
preferredequity = mergedBsIsSum.CapitalStock-mergedBsIsSum.CommonStock

In [73]:
#preferredequity

In [74]:
networth = mergedBsIsSum.InvestedCapital+cash-debt-preferredequity

In [88]:
valuationMeasures.loc['MET']

KeyError: 'MET'

In [76]:
faustmannRatio = mergedBsIsSum.MarketCap/networth

In [77]:
faustmannRatio.sort_values(ascending=False)

symbol
TEAM     80.575385
CL       67.920002
ALNY     54.260758
AMZN     32.330169
CLX      27.942194
           ...    
CMG     -61.319632
COR     -77.191674
CDK    -115.720703
ALK    -151.098909
CTXS   -162.003643
Length: 232, dtype: float64

In [78]:
#(roic/faustmannRatio).sort_values(ascending=False)

In [79]:
#criteria: ROIC > 20%

In [80]:
dfRoic = roic[roic>0.01].to_frame()
dfFaustmannRatio = faustmannRatio[faustmannRatio>0].to_frame()
dfRoicToFaustmann = (roic/faustmannRatio).to_frame()

In [81]:
dfRoic.columns=['roic']
dfFaustmannRatio.columns=['faustmannRatio']
dfRoicToFaustmann.columns=['roicToFaustmann']

In [82]:
dfFaustmannRatio.sort_values(by=['symbol'])

,faustmannRatio
symbol,
A,7.882902
AA,1.508937
AAP,7.625060
AAPL,20.314026
ABBV,5.402491
...,...
TWOU,2.458974
UHAL,10.319343
WRB,1.689745


In [83]:
dfFaustmannRatio['faustmannRank'] = dfFaustmannRatio.rank(ascending=True)
dfFaustmannRatio.sort_values(by=['faustmannRank'])

,faustmannRatio,faustmannRank
symbol,,
BHF,0.212785,1.0
BK,0.243204,2.0
C,0.300990,3.0
CNDT,0.439650,4.0
ALLY,0.455614,5.0
...,...,...
CLX,27.942194,208.0
AMZN,32.330169,209.0
ALNY,54.260758,210.0


In [84]:
dfRoic['roicRank']=dfRoic.rank(ascending=False)
dfRoic.sort_values(by=['roicRank'])

,roic,roicRank
symbol,,
AZO,0.636396,1.0
AAPL,0.504679,2.0
BIO,0.498231,3.0
CL,0.459012,4.0
CHE,0.442016,5.0
...,...,...
ADNT,0.029028,198.0
CDAY,0.024114,199.0
ADT,0.021871,200.0


In [85]:
result = pd.merge(
    pd.merge(
        pd.merge(mergedBsIsSum,dfRoic,on=['symbol']),
        dfFaustmannRatio,
        on=['symbol']
    ),
    dfRoicToFaustmann,
    on=['symbol']
)
#.sort_values(['roicToFaustmann'],ascending=False)

In [86]:
result['sumRanks']=result['faustmannRank']+result['roicRank']
result.sort_values(by=['sumRanks'],ascending=True)

,asOfDate,InvestedCapital,CashAndCashEquivalents,CurrentDeferredLiabilities,LongTermDebtAndCapitalLeaseObligation,NonCurrentDeferredLiabilities,OtherNonCurrentLiabilities,CapitalStock,CommonStock,EBIT,MarketCap,roic,roicRank,faustmannRatio,faustmannRank,roicToFaustmann,sumRanks
symbol,,,,,,,,,,,,,,,,,
AA,2020-12-31,7.191000e+09,1.607000e+09,0.0,2.545000e+09,1.020000e+08,8.400000e+07,2.000000e+06,2000000.0,1.719000e+09,9.154719e+09,0.239049,32.0,1.508937,47.0,0.158422,79.0
CNX,2020-12-31,6.914989e+09,5.091670e+08,17693000.0,2.872615e+09,4.761080e+08,1.597870e+08,2.241000e+06,2241000.0,1.244602e+09,3.237875e+09,0.179986,69.0,0.830660,17.0,0.216678,86.0
BIO,2020-12-31,9.893996e+09,6.622050e+08,42468000.0,4.389370e+08,2.076785e+09,1.969710e+08,3.000000e+03,3000.0,4.929499e+09,2.333109e+10,0.498231,3.0,2.990767,91.0,0.166590,94.0
EQH,2020-12-31,1.951500e+10,6.179000e+09,0.0,4.428000e+09,7.490000e+08,0.000000e+00,1.274000e+09,6000000.0,2.692000e+09,1.397608e+10,0.137945,90.0,0.726068,14.0,0.189989,104.0
HRB,2021-04-30,3.566298e+09,2.661914e+09,204976000.0,3.158439e+09,4.089780e+08,0.000000e+00,2.462000e+06,2462000.0,7.756060e+08,4.107731e+09,0.217482,48.0,1.672652,56.0,0.130022,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BKI,2020-12-31,4.822600e+09,3.470000e+07,59600000.0,2.121900e+09,3.767000e+08,9.490000e+07,0.000000e+00,0.0,2.882000e+08,1.387225e+10,0.059760,179.0,6.293553,147.0,0.009495,326.0
ADT,2020-12-31,1.422710e+10,3.631770e+08,345582000.0,1.012113e+10,1.886597e+09,5.106630e+08,8.257000e+06,8257000.0,3.111550e+08,8.967077e+09,0.021871,200.0,5.194361,133.0,0.004210,333.0
CHTR,2020-12-31,1.105230e+11,3.483000e+09,494000000.0,8.174400e+10,1.810800e+10,4.198000e+09,0.000000e+00,0.0,8.150000e+09,1.337417e+11,0.073740,155.0,14.134610,190.0,0.005217,345.0


In [87]:
result.sort_values(by=['sumRanks'],ascending=True).to_csv('austrian.csv', sep='\t')